# Создание пользовательской категории акций
## Загрузка данных
Источник данных: Тинькофф брокер API.

Для получения информации достаточно сгенерировать у брокера read only Bearer Token.

In [1]:
import requests
import json
import time
from tqdm import tqdm

In [36]:
DELAY = 1 # for requests, in seconds

# bearerToken from Tinkoff
bearerToken = 't.3zMPmkdCXwbvK4v48…И Т.Д. — НАДО ПОДСТАВИТЬ ДЕЙСТВУЮЩИЙ ТОКЕН'

urlShares = 'https://invest-public-api.tinkoff.ru/rest/tinkoff.public.invest.api.contract.v1.InstrumentsService/Shares'
urlGetAssetBy = 'https://invest-public-api.tinkoff.ru/rest/tinkoff.public.invest.api.contract.v1.InstrumentsService/GetAssetBy'

headers = {
    'Accept': 'application/json',
    'Authorization': 'Bearer ' + bearerToken,
    'Content-Type': 'application/json'
}

dataJsonForLoadShares = {
    'instrumentStatus': 'INSTRUMENT_STATUS_UNSPECIFIED'
}


In [37]:
def saveCSVHeadersToFile(f, columns):
    headersString = ''
    for c in columns:
        headersString += c + ','

    headersString = headersString[:-1]
    f.write(headersString + '\n')


def testShare(share):
    if (share['countryOfRisk'] == 'RU' 
        and share['forQualInvestorFlag'] == False
        and share['liquidityFlag'] == True
        and share['currency'] == 'rub'):
        return True

    return False


def postGetAssetBy(urlGetAssetBy, headers, dataJson):
    #print('POST call:', urlGetAssetBy)
    response = requests.post(urlGetAssetBy, headers=headers, data=json.dumps(dataJson))

    if response.status_code == 200:
        time.sleep(DELAY)
        return response.json()['asset']

    return { 'isError': True }


def makeRequestForGetAssetBy(idAsStr):
    return { 'id': idAsStr }

def filterText(txt):
    return txt.replace('\n', ' ')

def loadShares(urlShares, headers, dataJson, outputFileName):
    print('POST call:', urlShares)
    response = requests.post(urlShares, headers=headers, data=json.dumps(dataJson))

    if response.status_code == 200:  # HTTP status code 200 means success
        print('POST call is ok.')
        # Save response to file
        with open(outputFileName, 'w', encoding='utf-8') as fileTrain:
                instruments = response.json().get('instruments')
                saveCSVHeadersToFile(fileTrain, ('label', 'text'))

                for it in tqdm(instruments):
                    if (testShare(it)):

                        #print(it['ticker'])

                        asset = postGetAssetBy(urlGetAssetBy, 
                                               headers, 
                                               makeRequestForGetAssetBy(it['assetUid']))
                        
                        assetDescription = filterText(asset['description'])
                        brandDescription = filterText(asset['brand']['description'])

                        fileTrain.write('?,')

                        line = it['sector'] + ' ' + assetDescription + ' ' + brandDescription
                        line = line.replace('"', '')
                        fileTrain.write('"' + line + '"\n')
                        
    else:
        print('Error ' + str(response.status_code))

    print('Data loading is finished.')

In [38]:
loadShares(urlShares, headers, dataJsonForLoadShares, 'shares_dataset.csv')

POST call: https://invest-public-api.tinkoff.ru/rest/tinkoff.public.invest.api.contract.v1.InstrumentsService/Shares
POST call is ok.


100%|██████████| 1983/1983 [02:19<00:00, 14.25it/s]

Data loading is finished.


## Результат

Набор данных сохранён в файл shares_dataset.csv, колонка label заполнена знаками вопроса для заполнения меток (1 — позитивный класс, 0 — негативный) пользователем.